In [1]:
#Puxando API para df_bruto V2
import time
import requests
import pandas as pd
from tqdm import tqdm

ids = pd.read_excel('IDs.xlsx')
df_bruto = pd.DataFrame(columns=['id', 'json'])
pbar = tqdm(total=len(ids), position=0, leave=True)

for id in ids['entrada_id']:
    pbar.update()
    for attempt in range(5):  # Tenta até 5 vezes
        try:
            response = requests.get(f'https://api.seubone.com/layout/layouts/{id}')
            if response.status_code == 200:
                dados = response.json()
                df_bruto.loc[len(df_bruto)] = [id, dados]
                break
            else:
                print(f"Erro ao acessar o ID {id}: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição para ID {id}: {e}")
            time.sleep(2 ** attempt)  # Espera exponencial antes de tentar novamente
        time.sleep(0.5)  # Adiciona um atraso de 0,5 segundos entre as requisições

display(df_bruto)
df_bruto.to_excel('df_bruto.xlsx',index=False)


100%|██████████| 149/149 [01:39<00:00,  1.68it/s]

,id,json
0,80784,"{'id_entrada': 80790, 'id_layouts': 80784, 'no..."
1,80615,"{'id_entrada': 80621, 'id_layouts': 80615, 'no..."
2,80614,"{'id_entrada': 80620, 'id_layouts': 80614, 'no..."
3,80616,"{'id_entrada': 80622, 'id_layouts': 80616, 'no..."
4,80701,"{'id_entrada': 80707, 'id_layouts': 80701, 'no..."
...,...,...
144,80781,"{'id_entrada': 80787, 'id_layouts': 80781, 'no..."
145,80657,"{'id_entrada': 80663, 'id_layouts': 80657, 'no..."
146,80782,"{'id_entrada': 80788, 'id_layouts': 80782, 'no..."
147,80692,"{'id_entrada': 80698, 'id_layouts': 80692, 'no..."


In [2]:
#Criando df_clientes
df_clientes = pd.DataFrame(columns=['id_entrada','nome'])

for json in df_bruto['json']:
    id = json['id_entrada']
    nome = json['nome_cliente']
    df_clientes.loc[len(df_clientes)] = [id,nome]
    
display(df_clientes)
    

,id_entrada,nome
0,80790,Eduardo
1,80621,Lock Smith
2,80620,Rogerio saraiva
3,80622,Inara
4,80707,Anderson
...,...,...
144,80787,Diego Cabral 9510 +30Recompra Com Logo (Out...
145,80663,Enttony
146,80788,Ricardo Coelho
147,80698,Micheline


In [3]:
#Criando df_vendas e df_vendas_sem_orcamento
df_vendas = pd.DataFrame(columns=['id_entrada','id_layouts_api','orcamento_id','cidade','layout_id','preco unitario','qtd','modelo'])
df_clientes = pd.DataFrame(columns=['id_entrada','nome','cnpj','razao_social','cidade'])
df_vendas_sem_orcamento = pd.DataFrame(columns=['id_entrada','cliente'])


for json in df_bruto['json']:
    id_entrada = json['id_entrada']
    nome = json['nome_cliente']
    if len(json['orcamentos']) != 0:        
        id_layouts = json['id_layouts']
        cidade = json['orcamentos']['cidade']
        cnpj = json['orcamentos']['cnpj']
        razaoSocial = json['orcamentos']['razaoSocial']
        df_clientes.loc[len(df_clientes)] = [id_entrada,nome,cnpj,razaoSocial,cidade]
        for x in range(len(json['orcamentos']['orcamento_skus'])):
            preco_descontado = (json['orcamentos']['orcamento_skus'][x]['preco_descontado'])
            quantidade = (json['orcamentos']['orcamento_skus'][x]['quantidade'])
            layout_id = (json['orcamentos']['orcamento_skus'][x]['layout_id'])
            orcamento_id = (json['orcamentos']['orcamento_skus'][x]['orcamento_id'])
            modelo  = (json['orcamentos']['orcamento_skus'][x]['produto'])
            modelo = modelo.split(",")[0]
            
            
            df_vendas.loc[len(df_vendas)] = [id_entrada,id_layouts,orcamento_id,cidade,layout_id,preco_descontado,quantidade,modelo] 
    else:
         df_clientes.loc[len(df_clientes)] = [id_entrada,nome,'','','']
    
        

display(df_vendas)
display(df_clientes)


,id_entrada,id_layouts_api,orcamento_id,cidade,layout_id,preco unitario,qtd,modelo
0,80707,80701,14479,Salvador,479135,33.9,0,Trucker
1,80707,80701,14479,Salvador,479136,29.9,0,Trucker
2,80707,80701,14479,Salvador,479294,29.9,50,Trucker
3,80708,80702,14478,Rio de Janeiro,479197,35.9,0,New York
4,80708,80702,14478,Rio de Janeiro,479198,35.9,0,New York
...,...,...,...,...,...,...,...,...
207,80787,80781,14516,orlandia,479821,31.9,25,Trucker
208,80787,80781,14516,orlandia,480306,39.9,0,Chapéu Agro
209,80787,80781,14516,orlandia,481065,31.9,0,Trucker
210,80787,80781,14516,orlandia,0,41.9,30,Chapéu Agro


,id_entrada,nome,cnpj,razao_social,cidade
0,80790,Eduardo,,,
1,80621,Lock Smith,,,
2,80620,Rogerio saraiva,,,
3,80622,Inara,,,
4,80707,Anderson,77937660597,Anderson Almeida,Salvador
...,...,...,...,...,...
144,80787,Diego Cabral 9510 +30Recompra Com Logo (Out...,46071987000136,BERTOTTO ENGENHARIA E ASSESSORIA RURAL LTDA,orlandia
145,80663,Enttony,54851637000110,Enttony Felipe Pereira Cavalcante,Parauapebas
146,80788,Ricardo Coelho,,,
147,80698,Micheline,,,


In [4]:
#Criando df_modelos

df_modelos = pd.DataFrame(columns=['id_modelo','modelo'])

for json in df_bruto['json']:
    for i in range(len(json['modelos'])):
        try:
            id = json['modelos'][i]['id']
        except KeyError:
            id = ""
        try:
            modelo = json['modelos'][i]['modelo']
        except KeyError:
            modelo = ""
        df_modelos.loc[len(df_modelos)] = [id,modelo]

display(df_modelos)

,id_modelo,modelo
0,479767,Trucker
1,479768,Trucker
2,479769,Trucker
3,479770,Americano
4,479771,Americano
...,...,...
1056,478866,Bucket
1057,478869,Bucket
1058,478870,Trucker
1059,478871,Trucker


In [5]:
#criando view_vendas
view_vendas = df_vendas[:]
view_vendas = view_vendas.loc[view_vendas['qtd'] != 0]
view_vendas['valor_total'] = view_vendas['preco unitario'] * view_vendas['qtd']
view_vendas['modelo'] = view_vendas['modelo'].replace("Trucker LB - 100 a 299 unidades", "Trucker")
view_vendas = view_vendas.drop(columns=['cidade','id_layouts_api'])

display(view_vendas)



,id_entrada,orcamento_id,layout_id,preco unitario,qtd,modelo,valor_total
2,80707,14479,479294,29.9,50,Trucker,1495.0
5,80708,14478,45917,34.9,200,Dad Hat,6980.0
7,80624,14791,7,29.9,30,Trucker,897.0
17,80626,14976,2433,31.9,30,Modelo Sport,957.0
18,80628,14498,478775,24.9,150,Trucker,3735.0
31,80631,14607,478653,31.9,30,Trucker,957.0
46,80635,15128,760000000,32.9,50,Americano,1645.0
47,80635,15128,2147483647,31.9,50,Trucker,1595.0
48,80635,15128,1,16.9,3,Porta Longneck (355ml,50.7
52,80638,14735,0,35.9,30,New York,1077.0


In [6]:
#criando view_clientes
view_cliente = df_clientes[:]
view_cliente = view_cliente.loc[view_cliente['razao_social'] != ""]
display(view_cliente)




,id_entrada,nome,cnpj,razao_social,cidade
4,80707,Anderson,77937660597,Anderson Almeida,Salvador
6,80708,Luis 4062 (4),04121485000122,IGREJA BATISTA ATITUDE DA BARRA DA TIJUCA,Rio de Janeiro
7,80624,Daniela,17165982000174,Equilíbrio Dani Endócrino,Itajubá
9,80626,Paula Gasparini,33354695000169,Paula Rogina Gouvêa Gasparini,Vila Velha
11,80628,Fernanda 8014 REC,91332834353,Ezequiel Sisnando Xenofonte Neto,São Luís
13,80631,Jessica,34156867000152,JW comércio de móveis e decoração,Rio Bonito
15,80635,METALURGICA 4591 - Cadência inativo,73587404000105,METALÚRGICA RULENSKY LTDA ME,Blumenau
17,80638,Jairo Lucheta,51635699000105,LE TOUR INDOOR LTDA,Ribeirão Preto
26,80719,Vânia Mendes 5161 REC,39304681000117,A Igreja do Brasil,Juiz de Fora
34,80724,Anna Luisa 7264,39806720000184,PROMOVE ART BRINDES E COMUNICACAO EM TECIDOS LTDA,São Paulo


In [7]:
#criando view_modelos

view_modelos = df_modelos[:]
view_modelos = view_modelos.drop_duplicates(subset=['id_modelo'])
display(view_modelos)

,id_modelo,modelo
0,479767,Trucker
1,479768,Trucker
2,479769,Trucker
3,479770,Americano
4,479771,Americano
...,...,...
1056,478866,Bucket
1057,478869,Bucket
1058,478870,Trucker
1059,478871,Trucker


In [ ]:
view_total_vendas = view_vendas[:]
view_total_vendas = view_total_vendas[['id_entrada','valor_total']]
view_total_vendas = view_total_vendas.groupby('id_entrada').sum()
view_total_vendas['valor_total'].mean()

2429.05

In [ ]:
view_vendas.to_excel('view_vendas.xlsx',index=False)
view_cliente.to_excel('view_clientes.xlsx',index=False)
view_total_vendas.to_excel('view_total_vendas.xlsx')